### **1. Importar librerías necesarias**

In [1]:
import pandas as pd
import yfinance as yf
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go

### **2. Obtener datos históricos del BTC**
En esta sección, vamos a obtener los datos históricos del Bitcoin (BTC) utilizando la biblioteca `yfinance`. Esta biblioteca nos permite acceder a datos financieros históricos de diversas fuentes, incluyendo Yahoo Finance. Vamos a descargar los datos del BTC en dólares estadounidenses (USD) y almacenarlos en un DataFrame de `pandas` para su posterior análisis.

In [2]:
def importar_base_bitcoin():
  global df_bitcoin

  #Obtenemos datos historicos del Bitcoin
  datos = yf.download('BTC-USD', period= '7d', interval='5m', auto_adjust=True)

  # Generar el dataframe con los datos
  df_bitcoin = pd.DataFrame(datos)

  # Formatear columnas del dataframe
  df_bitcoin.columns = pd.Index(['Close', 'High', 'Low', 'Open', 'Volume'])
  df_bitcoin.index.name = None

In [3]:
importar_base_bitcoin()
df_bitcoin.head()

[*********************100%***********************]  1 of 1 completed


,Close,High,Low,Open,Volume
2025-08-07 00:00:00+00:00,114998.429688,114998.429688,114989.929688,114989.929688,0
2025-08-07 00:05:00+00:00,114941.742188,114984.375000,114941.742188,114976.804688,0
2025-08-07 00:10:00+00:00,114945.859375,114945.859375,114928.679688,114929.695312,0
2025-08-07 00:15:00+00:00,114952.914062,114969.578125,114952.914062,114964.375000,0
2025-08-07 00:20:00+00:00,114900.859375,114938.046875,114900.859375,114938.046875,935288832


### **3. Obtener tendencia del BTC**
En esta sección, vamos a obtener la tendencia y el precio actual del Bitcoin (BTC) en dólares estadounidenses (USD) utilizando técnicas de Web Scraping. Para ello, utilizaremos la biblioteca `requests` para realizar solicitudes HTTP y `BeautifulSoup` para analizar el contenido HTML de la página web de [CoinMarketCap](https://coinmarketcap.com/).
El objetivo es extraer la información relevante sobre el precio actual del BTC y su tendencia, lo que nos permitirá tener una referencia adicional al análisis de los datos históricos obtenidos en la sección anterior.


In [4]:
def extraer_tendencias():

  global precio_actual, tendencia
  url = 'https://coinmarketcap.com/'
  respuesta = requests.get(url)

  #Confirmación de que la busqueda se realizo correctamente
  if respuesta.status_code == 200:
    s = BeautifulSoup(respuesta.content, 'lxml')

    #Obtenemos los span de la primera fila de la tabla en coinmarketcap.com
    html_row = s.find_all("tr")
    html_row = html_row[1]
    elementos = html_row.find_all("span")

    #Obtenemos el precio actual del Bitcoin en dolares (USD)
    precio_actual = float(elementos[1].text.replace('$', '').replace(',',''))

    #Obtenemos la tendencia (Alta o Baja)
    if elementos[2].next.attrs['class'][0] == "icon-Caret-up":
      tendencia = 'Alta'
    else:
      tendencia = 'Baja'

  else:
    print("Error al cargar la página de CoinMarketCap")

In [5]:
extraer_tendencias()
print(f'El precio actual del Bitcoin es: ${precio_actual}')
print(f'La tendencia del Bitcoin es: {tendencia}')

El precio actual del Bitcoin es: $121992.1
La tendencia del Bitcoin es: Alta


### **4. Simplicar DataFrame para analizar**
En esta sección, vamos a simplificar el DataFrame obtenido en la sección anterior para facilitar su análisis. El objetivo es reducir la cantidad de columnas y filas, manteniendo solo la información esencial que necesitamos para nuestro análisis de trading algorítmico.

In [6]:
def resumen_df_bitcoin():
  global df_resumen

  df_resumen = df_bitcoin.copy()

  df_resumen.drop(columns=['High', 'Low', 'Open'], inplace=True)

  # Eliminar registros de 'Volume' con valor menor o igual a 0
  df_resumen.drop(df_resumen[df_resumen.Volume <= 0].index, inplace=True)

  # Eliminar indices repetidos
  df_resumen.drop_duplicates(inplace=True)

In [7]:
resumen_df_bitcoin()
df_resumen.head()

,Close,Volume
2025-08-07 00:20:00+00:00,114900.859375,935288832
2025-08-07 00:35:00+00:00,114973.531250,1119502336
2025-08-07 01:25:00+00:00,115184.976562,17031168
2025-08-07 01:35:00+00:00,115054.960938,1350922240
2025-08-07 01:55:00+00:00,114724.812500,77770752


In [8]:
def analisis_df_bitcoin():

  # Identificación de índices repetidos
  indices_repetidos = df_resumen.duplicated().sum()
  # Chequeo de valores nulos en la columna "Close
  valores_nulos_close = "Sí" if df_resumen.Close.isna().any() else "No"
  # Conteo de registros con Volume menor o igual a cero
  registros_cero = df_resumen.Volume[df_resumen.Volume <= 0].count()

  print(f'Existen {indices_repetidos} indices repetidos.')
  print(f"Existen valores nulos en la columna 'Close': {valores_nulos_close}")
  print(f"Registros que tengan un 'Volume' de transacción menor o igual a cero: {registros_cero}")

In [9]:
analisis_df_bitcoin()

Existen 0 indices repetidos.
Existen valores nulos en la columna 'Close': No
Registros que tengan un 'Volume' de transacción menor o igual a cero: 0


### **5. Filtrar y calcular media de precio BTC**
En esta sección, vamos a filtrar el DataFrame simplificado para centrarnos únicamente en los precios de cierre del Bitcoin (BTC) y calcular la media de estos precios. Este paso es nos permite obtener una visión general del comportamiento del precio de BTC en el período analizado.

In [10]:
def filtro_media_bitcoin():
  global df_filtro_bitcoin, media_bitcoin

  Q1 = df_resumen.quantile(0.25)
  Q3 = df_resumen.quantile(0.75)

  df_filtro_bitcoin = df_resumen[(df_resumen.Close >= Q1.Close) & (df_resumen.Close <= Q3.Close)]

  media_bitcoin = df_filtro_bitcoin.Close.mean()

In [11]:
filtro_media_bitcoin()
df_filtro_bitcoin.describe()

,Close,Volume
count,482.000000,4.820000e+02
mean,118089.197663,5.638455e+08
std,952.423691,5.155478e+08
min,116693.562500,5.734400e+05
25%,117079.533203,9.167872e+07
50%,118327.050781,3.784335e+08
75%,118906.705078,1.028556e+09
max,119575.031250,2.447196e+09


### **6. Toma de decisiones de trading**
En esta sección, vamos a implementar la lógica de trading basada en el análisis de los datos obtenidos. Utilizaremos la media de los precios de cierre del BTC para determinar si debemos comprar, vender o mantener nuestra posición. La estrategia se basará en comparar el precio actual con la media calculada y tomar decisiones informadas para maximizar nuestras oportunidades de trading.

In [12]:
def tomar_decision():
  global decision

  if (precio_actual >= media_bitcoin) and (tendencia == 'Baja'):
    decision = 'Vender'
  elif (precio_actual <= media_bitcoin) and (tendencia == 'Alta'):
    decision = 'Comprar'
  else:
    decision = 'Esperar'

In [13]:
tomar_decision()
print(f'La decisión a tomar es: {decision}')

La decisión a tomar es: Esperar


### **7. Visualización de precios y decisiones de trading**
En esta sección, vamos a visualizar los precios de cierre del Bitcoin (BTC) junto con las decisiones de trading tomadas en la sección anterior. Utilizaremos la biblioteca `plotly` para crear gráficos interactivos que nos permitan observar el comportamiento del precio a lo largo del tiempo y las decisiones de compra o venta en función de la media calculada.

In [14]:
def grafico_btc():
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=df_resumen.index, y=df_resumen.Close, name='Precio Bitcoin'))
  fig.add_trace(go.Scatter(x=df_resumen.index, y=[media_bitcoin]*len(df_resumen), name='Precio Promedio (7d)'))
  fig.update_layout(title='Precio del Bitcoin los ultimos 7 días', xaxis_title='Fecha', yaxis_title='Precio (USD)')
  fig.update_xaxes(rangeslider_visible=True)
  color = 'green' if decision == 'Vender' else 'blue' if decision == 'Comprar' else 'red'
  fig.add_annotation(
      x=df_resumen.index[-1],
      y=df_resumen.Close.iloc[-1],
      text=f'{decision}',
      showarrow=True,
      font=dict(size=12, color=color),
      arrowhead=2,
      arrowsize=1,
      arrowwidth=2,
      arrowcolor=color,
      ax=0,
      ay=-50,
      bordercolor=color,
      borderwidth=2,
      borderpad=4,
      bgcolor='white',
      opacity=0.8
  )
  fig.show()

In [18]:
grafico_btc()

In [16]:
print(f'El precio actual del Bitcoin es: ${precio_actual}')
print(f'La tendencia del Bitcoin es: {tendencia}')

El precio actual del Bitcoin es: $121992.1
La tendencia del Bitcoin es: Alta


### **8. Automatización del proceso de trading**
En esta sección, vamos a automatizar el proceso de trading utilizando los métodos y estrategias desarrollados en las secciones anteriores. Implementaremos un bucle que se ejecutará cada 5 minutos para actualizar los datos, calcular la media de precios y tomar decisiones de trading basadas en la lógica definida.

In [19]:
from IPython.display import clear_output
import time

try:
    while True:
        clear_output()
        importar_base_bitcoin()
        extraer_tendencias()
        resumen_df_bitcoin()
        filtro_media_bitcoin()
        tomar_decision()
        grafico_btc()
        time.sleep(300)
except KeyboardInterrupt:
    print("Ejecución detenida por el usuario.")

[*********************100%***********************]  1 of 1 completed


Ejecución detenida por el usuario.
